In [1]:
from typing import List, Literal, Tuple
import itertools
import random
from collections import defaultdict

from PIL import Image
from pprint import pprint

In [2]:
from image_composer import ImageComposer
#from image_generator import ImageGenerater
from storyboard_visualizer import StoryBoard

## image composer

In [3]:
VERTICAL_POSITIONING = {'Logo': [1], 'CTA Button': [1, 2, 3], 'Icon': [1, 2, 3], 'Product Image': [2],
               'Text Elements': [1,3], 'Infographic': [2], 'Banner': [1], 'Illustration': [2], 'Photograph': [2],
               'Mascot': [2], 'Testimonial Quotes': [2], 'Social Proof': [2, 1, 3], 'Seal or Badge': [3, 1, 2],
               'Graphs and Charts': [2], 'Decorative Elements': [3], 'Interactive Elements': [2],
               'Animation': [2], 'Coupon or Offer Code': [3], 'Legal Disclaimers or Terms': [3],
               'Contact Information': [3, 1, 2], 'Map or Location Image': [3], 'QR Code': [3, 1, 2]}

HORIZONTAL_POSITIONING = {'Logo': [1], 'CTA Button': [2, 1, 3], 'Icon': [1], 'Product Image': [1],
                          'Text Elements': [1], 'Infographic': [1], 'Banner': [2], 'Illustration': [2],
                          'Photograph': [2], 'Mascot': [1], 'Testimonial Quotes': [2], 'Social Proof': [3, 1, 2],
                          'Seal or Badge': [3, 1, 2], 'Graphs and Charts': [1], 'Decorative Elements': [3],
                          'Interactive Elements': [2], 'Animation': [2], 'Coupon or Offer Code': [3],
                          'Legal Disclaimers or Terms': [3], 'Contact Information': [3, 1, 2],
                          'Map or Location Image': [3], 'QR Code': [3, 1, 2]}


In [5]:
class ImageComposer:
    categories = Literal["Background", "Logo", "CTA Button", "Icon", "Product Image", "Text Elements", "Infographic", "Banner", "Illustration", "Photograph", "Mascot", "Testimonial Quotes", "Social Proof", "Seal or Badge", "Graphs and Charts", "Decorative Elements", "Interactive Elements", "Animation", "Coupon or Offer Code", "Legal Disclaimers or Terms", "Contact Information", "Map or Location Image", "QR Code"]
    PositionSegment = Tuple[float, float]
    AlignmentPosition = Tuple[int, int]
    AlignmentPositions = List[AlignmentPosition]
    frame_images = List[Tuple[categories, str, str]]


    #initalization
    def __init__(self, width:int, height: int, frames: List[frame_images]) -> None:
        self.width = width
        self.height = height
        self.frames = frames
        self.segments = ImageComposer.get_image_position_segments(width, height)
        self.generated_frames = []

    def generate_frames(self):
        self.compose_frames()
        return self.generated_frames

    def compose_frames(self) -> None:
        self.generated_frames = []

        for frame in self.frames:
            # Separate Background
            placement_items = []
            for index, item in enumerate(frame):
                if item[0] == "Background":
                    background_index = index
                    continue
                placement_items.append(item)
            
            background = frame[background_index]

            possibilties = ImageComposer.compute_positions([item[0] for item in placement_items])
            identified_locations = ImageComposer.select_diverse_positions(possibilties)
            adjusted_positions = self.calculate_adjusted_element_positions(identified_locations)
            placement_values = [(x[2], *list(y.values())) for x, y in zip(placement_items, adjusted_positions)]
            # Construct Frame
            self.generated_frames.append(self.create_combined_image(background[2], placement_values))

    @staticmethod
    def compute_positions(elements: List[categories]) -> List[AlignmentPositions]:
        possible_positions = []

        # Iterate through each element to calculate its position combinations
        for element in elements:
            vertical_options = VERTICAL_POSITIONING[element]
            horizontal_options = HORIZONTAL_POSITIONING[element]
            combinations = list(itertools.product(vertical_options, horizontal_options))
            possible_positions.append(combinations)

        return possible_positions
    
    @staticmethod
    def select_diverse_positions(possible_positions: List[AlignmentPositions]) -> AlignmentPositions:
        position_frequency = defaultdict(int)

        def update_position_frequency(selected_position):
            position_frequency[selected_position] += 1

        selected_positions = []
        
        for positions in possible_positions:
            sorted_combinations = sorted(positions, key=lambda x: position_frequency[x])
            
            lowest_frequency = position_frequency[sorted_combinations[0]]
            lowest_freq_combinations = [pos for pos in sorted_combinations if position_frequency[pos] == lowest_frequency]
            
            selected_position = random.choice(lowest_freq_combinations)
            selected_positions.append(selected_position)
            
            update_position_frequency(selected_position)
        
        return selected_positions


    @staticmethod
    def get_image_position_segments(width: float, height: float, vm: float = 0.6, vo: float = 0.2, hm: float = 0.6, ho: float = 0.2) -> Tuple[List[PositionSegment], List[PositionSegment]]:
        """Divide Image based on percentage for vertical and horizontal segments."""
        
        if vm + vo * 2 > 1 or hm + ho * 2 > 1:
            raise ValueError("Sum of percentages exceeds 100% for either vertical or horizontal segments.")
        
        vertical_mid = height * vm
        vertical_outer = height * vo
        horizontal_mid = width * hm
        horizontal_outer = width * ho

        vertical_segments = [
            (0, vertical_outer),
            (vertical_outer, vertical_outer + vertical_mid),
            (vertical_outer + vertical_mid, height)
        ]
        
        horizontal_segments = [
            (0, horizontal_outer),
            (horizontal_outer, horizontal_outer + horizontal_mid),
            (horizontal_outer + horizontal_mid, width)
        ]

        segements = []
        for vs in vertical_segments:
            vs_items = []
            for hs in horizontal_segments:
                vs_items.append((vs, hs))
            segements.append(vs_items)


        return segements
    
    def calculate_adjusted_element_positions(self, elements_positions, padding=10):
        element_details = []
        segment_elements = {}

        # Organize elements by their segments
        for i, (v_pos, h_pos) in enumerate(elements_positions):
            segment_key = (v_pos, h_pos)
            if segment_key not in segment_elements:
                segment_elements[segment_key] = []
            segment_elements[segment_key].append(i)
        
        for segment_key, elements in segment_elements.items():
            v_pos, h_pos = segment_key
            segment = self.segments[v_pos-1][h_pos-1]
            vertical_segment, horizontal_segment = segment
            num_elements = len(elements)
            
            x_start, x_end = horizontal_segment
            y_start, y_end = vertical_segment
            segment_width = (x_end - x_start) - 2 * padding
            segment_height = (y_end - y_start) - 2 * padding
            
            # Determine alignment and divide space
            is_vertical = segment_height > segment_width
            if is_vertical:
                space_per_element = segment_height / num_elements
            else:
                space_per_element = segment_width / num_elements
            
            for index, _ in enumerate(elements):
                if is_vertical:
                    element_x_start = x_start + padding
                    element_y_start = y_start + padding + index * space_per_element
                    element_width = segment_width
                    element_height = space_per_element
                else:
                    element_x_start = x_start + padding + index * space_per_element
                    element_y_start = y_start + padding
                    element_width = space_per_element
                    element_height = segment_height
                
                element_details.append({
                    "start_point": (element_x_start, element_y_start),
                    "dimensions": (element_width, element_height)
                })

        return element_details
    
    @staticmethod
    def resize_image(image, target_width, target_height):
        """
        Resize an image to fit within target dimensions while maintaining aspect ratio.
        """
        original_width, original_height = image.size
        ratio = min(target_width / original_width, target_height / original_height)
        new_width = int(original_width * ratio)
        new_height = int(original_height * ratio)
        resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)
        return resized_image

    def create_combined_image(self, background_path: str, elements: List[Tuple[str, int|float, int|float]]) -> Image.Image:
        """
        Create a combined image based on background and elements' positioning and sizing.
        
        :param background_path: Path to the background image.
        :param elements: A list of dictionaries, each containing 'image_path', 'start_point', and 'dimensions'.
        """
        # Load the background image
        background = Image.open(background_path).convert("RGBA")
        
        for element in elements:
            # Load element image
            image_path = element[0]
            image = Image.open(image_path).convert("RGBA")
            
            # Resize image according to dimensions without losing aspect ratio
            target_width, target_height = element[2]
            resized_image = ImageComposer.resize_image(image, target_width, target_height)
            
            # Calculate position to center the image within its segment
            start_x, start_y = element[1]
            offset_x = start_x + (target_width - resized_image.size[0]) / 2
            offset_y = start_y + (target_height - resized_image.size[1]) / 2
            
            # Place the resized image on the background
            background.paste(resized_image, (int(offset_x), int(offset_y)), resized_image)
        
        return background
    

In [6]:
if __name__ == "__main__":
    ic = ImageComposer(320, 500, [[('Logo', 'url_path', 'local_path'), 
                                   ('CTA Button', 'url_path', 'local_path'),
                                   ('Icon', 'url_path', 'local_path'),
                                   ('Product Image', 'url_path', 'local_path'),
                                   ('Text Elements', 'url_path', 'local_path')]])
    possibilties = ImageComposer.compute_positions(["Logo", "CTA Button", "Icon", "Product Image", "Text Elements"])
    pprint(possibilties)
    print("======================================================")
    diverse = ImageComposer.select_diverse_positions(possibilties)
    pprint(diverse)

    print(ic.calculate_adjusted_element_positions(diverse))

[[(1, 1)],
 [(1, 2), (1, 1), (1, 3), (2, 2), (2, 1), (2, 3), (3, 2), (3, 1), (3, 3)],
 [(1, 1), (2, 1), (3, 1)],
 [(2, 1)],
 [(1, 1), (3, 1)]]
[(1, 1), (2, 3), (3, 1), (2, 1), (3, 1)]
[{'start_point': (10, 10.0), 'dimensions': (44.0, 80.0)}, {'start_point': (266.0, 110.0), 'dimensions': (44.0, 280.0)}, {'start_point': (10, 410.0), 'dimensions': (44.0, 40.0)}, {'start_point': (10, 450.0), 'dimensions': (44.0, 40.0)}, {'start_point': (10, 110.0), 'dimensions': (44.0, 280.0)}]


## Storyboard Visualizer

In [14]:
from typing import List

from PIL import Image

class StoryBoard:

    @staticmethod
    def combine_images_horizontally(images, separation_space=100, vertical_padding=200, background_color=(255, 255, 255)):
        """
        Combines multiple images into a new image, displayed horizontally on a larger background.
        Images are centered horizontally within the background and have vertical padding.

        :param images: loaded pillow images.
        :param separation_space: Space between images in pixels.
        :param vertical_padding: Vertical padding for the top and bottom of the images.
        :param background_color: Background color of the new image as an RGB tuple.
        :return: Combined image.
        """
        # images = [Image.open(path) for path in image_paths]
        widths, heights = zip(*(i.size for i in images))

        # Calculate total width and max height for the images, considering separation space
        total_images_width = sum(widths) + separation_space * (len(images) - 1)
        max_height = max(heights) + vertical_padding * 2

        # Calculate the background size
        background_width = total_images_width + vertical_padding * 2  # Padding on left and right for uniformity
        background_height = max_height

        # Create the background image
        background = Image.new('RGB', (background_width, background_height), color=background_color)

        # Calculate the starting x coordinate to center the images horizontally
        x_offset = (background_width - total_images_width) // 2

        # Paste each image, centered vertically
        for img in images:
            y_offset = (background_height - img.height) // 2
            background.paste(img, (x_offset, y_offset))
            x_offset += img.width + separation_space

        return background
    


In [15]:
from PIL import Image

# Define image paths
i1 = 'images/01d8fa51-ca7b-499d-9938-c13f3b496439.png'
i2 = 'images/48e6f2e6-b292-4cae-81aa-b0362cc803f1.png'
i3 = "images/d7c83396-f43f-4d61-bdf4-76db405bf2ef.png"

# Load image into Image objects
image1 = Image.open(i1)
image2 = Image.open(i2)
image3 = Image.open(i3)

# Display images
image1.show()
image2.show()
image3.show()


Opening in existing browser session.
Opening in existing browser session.


Opening in existing browser session.


In [16]:
if __name__ == "__main__":
    image = StoryBoard.combine_images_horizontally([image1,image2,image3])
    image.show()

Opening in existing browser session.


## update positioning
The VERTICAL_POSITIONING and HORIZONTAL_POSITIONING dictionaries now include additional positioning options like Top, Bottom, Left, Right, and Center.

In [20]:
VERTICAL_POSITIONING = {'Logo': [1], 'CTA Button': [1, 2, 3], 'Icon': [1, 2, 3], 'Product Image': [2],
               'Text Elements': [1,3], 'Infographic': [2], 'Banner': [1], 'Illustration': [2], 'Photograph': [2],
               'Mascot': [2], 'Testimonial Quotes': [2], 'Social Proof': [2, 1, 3], 'Seal or Badge': [3, 1, 2],
               'Graphs and Charts': [2], 'Decorative Elements': [3], 'Interactive Elements': [2],
               'Animation': [2], 'Coupon or Offer Code': [3], 'Legal Disclaimers or Terms': [3],
               'Contact Information': [3, 1, 2], 'Map or Location Image': [3], 'QR Code': [3, 1, 2],
               'Top': [1], 'Bottom': [3], 'Center': [2]}

HORIZONTAL_POSITIONING = {'Logo': [1], 'CTA Button': [2, 1, 3], 'Icon': [1], 'Product Image': [1],
                          'Text Elements': [1], 'Infographic': [1], 'Banner': [2], 'Illustration': [2],
                          'Photograph': [2], 'Mascot': [1], 'Testimonial Quotes': [2], 'Social Proof': [3, 1, 2],
                          'Seal or Badge': [3, 1, 2], 'Graphs and Charts': [1], 'Decorative Elements': [3],
                          'Interactive Elements': [2], 'Animation': [2], 'Coupon or Offer Code': [3],
                          'Legal Disclaimers or Terms': [3], 'Contact Information': [3, 1, 2],
                          'Map or Location Image': [3], 'QR Code': [3, 1, 2],
                          'Left': [1], 'Right': [3], 'Center': [2]}


In [21]:
import itertools
import random
from collections import defaultdict
from typing import List, Tuple
from PIL import Image

class ImageComposer:
    categories = Literal["Background", "Logo", "CTA Button", "Icon", "Product Image", "Text Elements", "Infographic", "Banner", "Illustration", "Photograph", "Mascot", "Testimonial Quotes", "Social Proof", "Seal or Badge", "Graphs and Charts", "Decorative Elements", "Interactive Elements", "Animation", "Coupon or Offer Code", "Legal Disclaimers or Terms", "Contact Information", "Map or Location Image", "QR Code"]
    PositionSegment = Tuple[float, float]
    AlignmentPosition = Tuple[int, int]
    AlignmentPositions = List[AlignmentPosition]
    frame_images = List[Tuple[categories, str, str]]

    def __init__(self, width:int, height: int, frames: List[frame_images]) -> None:
        self.width = width
        self.height = height
        self.frames = frames
        self.segments = ImageComposer.get_image_position_segments(width, height)
        self.generated_frames = []

    def generate_frames(self):
        self.compose_frames()
        return self.generated_frames

    def compose_frames(self) -> None:
        self.generated_frames = []

        for frame in self.frames:
            # Separate Background
            placement_items = []
            for index, item in enumerate(frame):
                if item[0] == "Background":
                    background_index = index
                    continue
                placement_items.append(item)
            
            background = frame[background_index]

            possibilties = ImageComposer.compute_positions([item[0] for item in placement_items])
            identified_locations = ImageComposer.select_diverse_positions(possibilties)
            adjusted_positions = self.calculate_adjusted_element_positions(identified_locations)
            placement_values = [(x[2], *list(y.values())) for x, y in zip(placement_items, adjusted_positions)]
            # Construct Frame
            self.generated_frames.append(self.create_combined_image(background[2], placement_values))

    @staticmethod
    def compute_positions(elements: List[categories]) -> List[AlignmentPositions]:
        possible_positions = []

        # Iterate through each element to calculate its position combinations
        for element in elements:
            vertical_options = VERTICAL_POSITIONING[element]
            horizontal_options = HORIZONTAL_POSITIONING[element]
            combinations = list(itertools.product(vertical_options, horizontal_options))
            possible_positions.append(combinations)

        return possible_positions
    
    @staticmethod
    def select_diverse_positions(possible_positions: List[AlignmentPositions]) -> AlignmentPositions:
        position_frequency = defaultdict(int)

        def update_position_frequency(selected_position):
            position_frequency[selected_position] += 1

        selected_positions = []
        
        for positions in possible_positions:
            sorted_combinations = sorted(positions, key=lambda x: position_frequency[x])
            
            lowest_frequency = position_frequency[sorted_combinations[0]]
            lowest_freq_combinations = [pos for pos in sorted_combinations if position_frequency[pos] == lowest_frequency]
            
            selected_position = random.choice(lowest_freq_combinations)
            selected_positions.append(selected_position)
            
            update_position_frequency(selected_position)
        
        return selected_positions


    @staticmethod
    def get_image_position_segments(width: float, height: float, vm: float = 0.6, vo: float = 0.2, hm: float = 0.6, ho: float = 0.2) -> Tuple[List[PositionSegment], List[PositionSegment]]:
        """Divide Image based on percentage for vertical and horizontal segments."""
        
        if vm + vo * 2 > 1 or hm + ho * 2 > 1:
            raise ValueError("Sum of percentages exceeds 100% for either vertical or horizontal segments.")
        
        vertical_mid = height * vm
        vertical_outer = height * vo
        horizontal_mid = width * hm
        horizontal_outer = width * ho

        vertical_segments = [
            (0, vertical_outer),
            (vertical_outer, vertical_outer + vertical_mid),
            (vertical_outer + vertical_mid, height)
        ]
        
        horizontal_segments = [
            (0, horizontal_outer),
            (horizontal_outer, horizontal_outer + horizontal_mid),
            (horizontal_outer + horizontal_mid, width)
        ]

        segements = []
        for vs in vertical_segments:
            vs_items = []
            for hs in horizontal_segments:
                vs_items.append((vs, hs))
            segements.append(vs_items)


        return segements
    
    def calculate_adjusted_element_positions(self, elements_positions, padding=10):
        element_details = []
        segment_elements = {}

        # Organize elements by their segments
        for i, (v_pos, h_pos) in enumerate(elements_positions):
            segment_key = (v_pos, h_pos)
            if segment_key not in segment_elements:
                segment_elements[segment_key] = []
            segment_elements[segment_key].append(i)
        
        for segment_key, elements in segment_elements.items():
            v_pos, h_pos = segment_key
            segment = self.segments[v_pos-1][h_pos-1]
            vertical_segment, horizontal_segment = segment
            num_elements = len(elements)
            
            x_start, x_end = horizontal_segment
            y_start, y_end = vertical_segment
            segment_width = (x_end - x_start) - 2 * padding
            segment_height = (y_end - y_start) - 2 * padding
            
            # Determine alignment and divide space
            is_vertical = segment_height > segment_width
            if is_vertical:
                space_per_element = segment_height / num_elements
            else:
                space_per_element = segment_width / num_elements
            
            for index, _ in enumerate(elements):
                if is_vertical:
                    element_x_start = x_start + padding
                    element_y_start = y_start + padding + index * space_per_element
                    element_width = segment_width
                    element_height = space_per_element
                else:
                    element_x_start = x_start + padding + index * space_per_element
                    element_y_start = y_start + padding
                    element_width = space_per_element
                    element_height = segment_height
                
                element_details.append({
                    "start_point": (element_x_start, element_y_start),
                    "dimensions": (element_width, element_height)
                })

        return element_details
    
    @staticmethod
    def resize_image(image, target_width, target_height):
        """
        Resize an image to fit within target dimensions while maintaining aspect ratio.
        """
        original_width, original_height = image.size
        ratio = min(target_width / original_width, target_height / original_height)
        new_width = int(original_width * ratio)
        new_height = int(original_height * ratio)
        resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)
        return resized_image

    def create_combined_image(self, background_path: str, elements: List[Tuple[str, int|float, int|float]]) -> Image.Image:
        """
        Create a combined image based on background and elements' positioning and sizing.
        
        :param background_path: Path to the background image.
        :param elements: List of elements with their path, start point, and dimensions.
        """
        background = Image.open(background_path).convert("RGBA")

        for element in elements:
            image_path = element[0]
            image = Image.open(image_path).convert("RGBA")
            
            target_width, target_height = element[2]
            resized_image = ImageComposer.resize_image(image, target_width, target_height)

            start_x, start_y = element[1]
            offset_x = start_x + (target_width - resized_image.size[0]) / 2
            offset_y = start_y + (target_height - resized_image.size[1]) / 2
            
            background.paste(resized_image, (int(offset_x), int(offset_y)), resized_image)
        
        return background


In [ ]:
if __name__ == "__main__":
    image = StoryBoard.combine_images_horizontally([image1,image2,image3])
    image.show()

In [22]:
from PIL import Image

class StoryBoard:
    @staticmethod
    def combine_images_horizontally(images: list[Image.Image]) -> Image.Image:
        """
        Combines a list of images into a single image horizontally.

        :param images: List of PIL Image objects.
        :return: Combined image.
        """
        # Calculate total width and max height
        total_width = sum(image.width for image in images)
        max_height = max(image.height for image in images)

        # Create a new blank image with calculated dimensions
        combined_image = Image.new('RGBA', (total_width, max_height))

        # Paste each image into the combined image
        current_x = 0
        for image in images:
            combined_image.paste(image, (current_x, 0))
            current_x += image.width

        return combined_image

    @staticmethod
    def combine_images_vertically(images: list[Image.Image]) -> Image.Image:
        """
        Combines a list of images into a single image vertically.

        :param images: List of PIL Image objects.
        :return: Combined image.
        """
        # Calculate total height and max width
        total_height = sum(image.height for image in images)
        max_width = max(image.width for image in images)

        # Create a new blank image with calculated dimensions
        combined_image = Image.new('RGBA', (max_width, total_height))

        # Paste each image into the combined image
        current_y = 0
        for image in images:
            combined_image.paste(image, (0, current_y))
            current_y += image.height

        return combined_image

    @staticmethod
    def create_storyboard(images: list[list[Image.Image]]) -> Image.Image:
        """
        Creates a storyboard by combining images both horizontally and vertically.

        :param images: A 2D list of PIL Image objects.
        :return: Storyboard image.
        """
        combined_rows = [StoryBoard.combine_images_horizontally(row) for row in images]
        storyboard = StoryBoard.combine_images_vertically(combined_rows)
        return storyboard

if __name__ == "__main__":
    # Define paths to images in the ./samples/ directory
    image1_path = "./samples/kfc-fs-320x480-sensoryvideo-storyboard.png"
    image2_path = "./samples/ITC-FS-320x480-SensorySwipe-Storyboard-Rev.png"
    image3_path = "./samples/Adludio-CocaCola-[BR]-[LIVE]-ifood-christmas2033-TapAndHold-FS-V3-sb.png"
    image4_path = "./samples/Adludio-Volvo-[UK]-[RFP]-Volvo_Vehicle_Electrification_XC40-Tap-FS-Version_2_AJ.png"
    image5_path = "./samples/Adludio-Microsoft-[FR]-[LIVE]-Windows_11_version_2-Swipe-MPU-v2.png"
    image6_path = "./samples/Disney-DrStrange-FS-600x900-UserSlider-Storyboard.png"

    # Open images using Pillow
    image1 = Image.open(image1_path)
    image2 = Image.open(image2_path)
    image3 = Image.open(image3_path)
    image4 = Image.open(image4_path)
    image5 = Image.open(image5_path)
    image6 = Image.open(image6_path)

    

    # Example 2D list of images to form a storyboard
    images = [
        [image1, image2, image3],  # First row
        [image4, image5, image6]   # Second row
    ]

    # Create storyboard
    storyboard = StoryBoard.create_storyboard(images)
    storyboard.show()


Opening in existing browser session.


### frame names extraction

In [19]:
import os

def list_filenames(directory):
    """
    List all filenames under a specified directory (including subdirectories).

    Args:
    - directory (str): Path to the directory.

    Returns:
    - filename_list (list): List of filenames (without paths) under the directory.
    """
    filename_list = []
    # Walk through the directory and its subdirectories
    for root, directories, files in os.walk(directory):
        for filename in files:
            filename_list.append(filename)
    return filename_list


if __name__ == "__main__":
    directory_path = '/home/tema/10X/Temp/tutorial/samples'
    filenames = list_filenames(directory_path)
    
    # Print all filenames
    for filename in filenames:
        print(filename)


Adludio-LEGO-[UK]-[RFP]-Ninjago24-Gamified-FlyingArcade-FS-V2-sb.png
nyle.png
bestuy-fs-600x900-tapandhold-storyboard.png
Amazon-ThirteenLives_FS_600x900-storyboard.png
Adludio-Detran_RS-[BR]-[LIVE]-Carnival_24-User_Choice_Quiz-FS-Version_2-Taxi.png
boa_FS_600x900_storyboard.png
kfc-fs-320x480-sensoryvideo-storyboard.png
ITC-FS-320x480-SensorySwipe-Storyboard-Rev.png
Meta_FS_320x480_swipe-storyboard.png
Adludio-CocaCola-[BR]-[LIVE]-ifood-christmas2033-TapAndHold-FS-V3-sb.png
Ladbroked_FS_320x480_spinwheel_-storyboard-uk.png
Adludio-Volvo-[UK]-[RFP]-Volvo_Vehicle_Electrification_XC40-Tap-FS-Version_2_AJ.png
Adludio-Microsoft-[FR]-[LIVE]-Windows_11_version_2-Swipe-MPU-v2.png
Disney-DrStrange-FS-600x900-UserSlider-Storyboard.png
